<a href="https://colab.research.google.com/github/Ljupka/Neural_Network_Bio/blob/main/04_DNA_enhancers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [5]:
!pip install -q genomic-benchmarks
!pip install torchmetrics -q

## Text preprocessing

In [8]:
import torch
example_seq = 'ACCCTGCCAACACGGGACTTTAC'
vocab = {'A':0,'C':1,'T':2,'G':3}

In [9]:
numericalized = [vocab[c] for c in example_seq]
numericalized

[0, 1, 1, 1, 2, 3, 1, 1, 0, 0, 1, 0, 1, 3, 3, 3, 0, 1, 2, 2, 2, 0, 1]

In [10]:
numericalized_tensor = torch.tensor(numericalized)
ohe_seq = torch.nn.functional.one_hot(numericalized_tensor, num_classes=5)
ohe_seq

tensor([[1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0]])

## Promoters Project

Your task is to

1.   **Create model for DNA sequence classification based on if it contains an promoter (label 1) or not (label 0)**
2.   **Show that your model is generalizing on new unseen data**

Tips
*   Use the pytorch documentation
*   You can use nn.Conv1d layer to perform convolution over 1D data.
*   Feel free to use any other improvements you can think of or find on the internet (e.g. more metrics, different architecture...)
*   Use GPU for training






In [11]:
from genomic_benchmarks.dataset_getters.pytorch_datasets import HumanNontataPromoters
import pandas as pd

train_df = pd.DataFrame(data=[{'x':x,'y':y} for x,y in HumanNontataPromoters('train')])
test_df = pd.DataFrame(data=[{'x':x,'y':y} for x,y in HumanNontataPromoters('test')])

/usr/local/lib/python3.12/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/usr/local/lib/python3.12/dist-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


In [67]:
train_df

,x,y
0,TTGCTTTCCATCGAACTCTGCAAATTTTGCAATAGGGGGAGGGATT...,0
1,AGTGTTTTCTTGTGGGGGCAGATAAAGGAAAAGGGAGGGGAACAAA...,0
2,CGGAAGGGCTGGAGCGTGTGCGGATCTCCGCGTCGGAGCTGCGCGG...,0
3,AACTGCTGTGTCACTACTCTGTGCTGGGTAAGAAAAAGGAAAACAA...,0
4,GATCCCTCGCAGCCCTCCCCCAGCCCAGGAGTAGTCGAGAGAGACT...,0
...,...,...
27092,ATGATTAAGATGTGGACAAGGTGAAGCCGATGGAGGGGGAGCTTTG...,1
27093,TTAGTATCAGATTGCGTCAGGGAGAAAGAAGGGTTATTAACATTAA...,1
27094,TCCAAATGTTGTGAATGGATTAAATTCTGCTTCCACTAGAAGCTTT...,1
27095,GGGGGGGCTGGCGCGAGGCTGCCTGGCTAGGCCTTGGCGTTCCCCC...,1


In [13]:
from itertools import product

letters = list(vocab.keys())
triplets = [''.join(p) for p in product(letters, repeat=3)]
vocab_components = {triplet: i for i, triplet in enumerate(triplets)}

In [14]:
def get_components(seq, component_length=3):
    return [seq[i:i+component_length] for i in range(len(seq) - component_length + 1)]

In [15]:
def bag_of_components(components, vocab_components):
    bow_vector = torch.zeros(len(vocab_components) + 1) # Add 1 for unknown components
    for comp in components:
        if comp in vocab_components:
            bow_vector[vocab_components[comp]] += 1
        else:
            bow_vector[len(vocab_components)] += 1 # Increment count for unknown components
    return bow_vector

In [24]:
from torch.utils.data import Dataset, DataLoader

class PromotersDataset(Dataset):
    def __init__(self, train_df):
       self.train_df = train_df.copy()

       train_df['components'] = train_df['x'].apply(get_components)
       train_df['bow_components'] = train_df['components'].apply(lambda x: bag_of_components(x, vocab_components))

    def __len__(self):
       return len(self.train_df)

    def __getitem__(self, idx):
        if idx < 0 or idx >= len(self):
            raise IndexError(f"Index {idx} out of range")
        row = self.train_df.iloc[idx]
        # return a dict; convert to tensors here if desired
        return {
            "x": row["x"],
            "components": row["components"],
            "bow_components": row["bow_components"],
            "y": row["y"]
        }

In [25]:
dataset = PromotersDataset(train_df)

In [26]:
print(len(dataset))
print(dataset[0])

27097
{'x': 'TGGGGTACCAGGAGGACGCGAGCTGACTACCTGACTCCGGGCGGCGCAGGACGCCCGCCGCGCGAGGGGCGGGGGCTAAGCGTATCGAGGGGCGGTGCTCGAGGGCGCGGGAGGCGGGAGAGGCTGCCGCGCCGTGTGCCCTGGCAGCCGCAGCCCCGCCCCGCCGCGTCCGGCCCGGTCCTGTCCCGCAGCGTCCCGCCAGCCAGCTCCTTGCACCCTTCGCGGCCGAGGCGCTCCCTGGTGCTCCCCGC', 'components': ['TGG', 'GGG', 'GGG', 'GGT', 'GTA', 'TAC', 'ACC', 'CCA', 'CAG', 'AGG', 'GGA', 'GAG', 'AGG', 'GGA', 'GAC', 'ACG', 'CGC', 'GCG', 'CGA', 'GAG', 'AGC', 'GCT', 'CTG', 'TGA', 'GAC', 'ACT', 'CTA', 'TAC', 'ACC', 'CCT', 'CTG', 'TGA', 'GAC', 'ACT', 'CTC', 'TCC', 'CCG', 'CGG', 'GGG', 'GGC', 'GCG', 'CGG', 'GGC', 'GCG', 'CGC', 'GCA', 'CAG', 'AGG', 'GGA', 'GAC', 'ACG', 'CGC', 'GCC', 'CCC', 'CCG', 'CGC', 'GCC', 'CCG', 'CGC', 'GCG', 'CGC', 'GCG', 'CGA', 'GAG', 'AGG', 'GGG', 'GGG', 'GGC', 'GCG', 'CGG', 'GGG', 'GGG', 'GGG', 'GGC', 'GCT', 'CTA', 'TAA', 'AAG', 'AGC', 'GCG', 'CGT', 'GTA', 'TAT', 'ATC', 'TCG', 'CGA', 'GAG', 'AGG', 'GGG', 'GGG', 'GGC', 'GCG', 'CGG', 'GGT', 'GTG', 'TGC', 'GCT', 'CTC', 'TCG', 'CGA', 'GAG', 'AGG', 'GGG

In [27]:
# Model
import torch.nn as nn
import math


class CNN(nn.Module):
    def __init__(self,num_classes=2):
        super().__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=4, stride=4),
            #nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2),
            #nn.ReLU(),
            #nn.MaxPool2d(kernel_size=4, stride=4),
        )
        # Flatten layer
        self.flatten = nn.Flatten()

        # Fully connected layers
        self.fc1 = nn.LazyLinear(128)
        self.relu3 = nn.ReLU()
        # self.fc2 = nn.Linear(128, num_classes)


    def forward(self, x):
        self.conv_layers(x)

model = CNN(num_classes=2)

In [35]:
model(torch.rand(1, 3, 1))

RuntimeError: Given input size: (1x16x1). Calculated output size: (1x4x0). Output size is too small

In [56]:
enumerate(train_loader)

In [78]:
import numpy as np
from torchmetrics import Accuracy

train_loader = torch.utils.data.DataLoader(dataset)

def train (model, dataset, gpu=False):

  accuracy_function = Accuracy(task='multiclass', num_classes=2)

  if(gpu):
    model.cuda()
    accuracy_function.cuda()

  num_epochs = 2

  for epoch in range(num_epochs):
    for batch_idx, inputs in enumerate(train_loader):

      sequence = inputs['x']
      bow_component = inputs['bow_components']
      label = inputs['y']

      print(inputs.shape)
      inputs = inputs.to('cuda')
      labels = labels.to('cuda')

      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      if (batch_idx) % 10 == 0:
        print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Accuracy: %.4f'
          %(epoch+1, num_epochs, batch_idx, len(train_loader.dataset)//inputs.size()[0], loss.item(), accuracy_function(outputs,labels)))




In [79]:
train(model, dataset)

AttributeError: 'dict' object has no attribute 'shape'

## Testing

In [ ]:
from tqdm import tqdm
from torchmetrics import Accuracy

def evaluate(model, dataset, gpu=True):
  accuracy_function = Accuracy(task='binary')

  if(gpu):
    model.cuda()
    accuracy_function.cuda()

    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    model.eval() #Turn off training-only layers
    all_predictions = []
    all_labels = []
    with torch.no_grad(): #Dont track gradients
      for batch_x,batch_y in tqdm(loader):
        if(gpu):
          batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

        output = model(batch_x)
        all_predictions.append(output)
        all_labels.append(batch_y)

    print('Accuracy:', accuracy_function(torch.cat(all_predictions), torch.cat(all_labels)).item())

In [ ]:
evaluate(model, train_dset)

In [ ]:
evaluate(model, test_dset)

In [65]:
from itertools import product

# Bases
bases = ['A', 'C', 'G', 'T']

# Generate all 3-mer combinations
all_3mers = [''.join(p) for p in product(bases, repeat=3)]

print("Total 3-mers:", len(all_3mers))
print(all_3mers)

Total 3-mers: 64
['AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']


In [66]:
encoded_codons = [[vocab[base] for base in all_3mer] for all_3mer in all_3mers]
print(encoded_codons)

[[0, 0, 0], [0, 0, 1], [0, 0, 3], [0, 0, 2], [0, 1, 0], [0, 1, 1], [0, 1, 3], [0, 1, 2], [0, 3, 0], [0, 3, 1], [0, 3, 3], [0, 3, 2], [0, 2, 0], [0, 2, 1], [0, 2, 3], [0, 2, 2], [1, 0, 0], [1, 0, 1], [1, 0, 3], [1, 0, 2], [1, 1, 0], [1, 1, 1], [1, 1, 3], [1, 1, 2], [1, 3, 0], [1, 3, 1], [1, 3, 3], [1, 3, 2], [1, 2, 0], [1, 2, 1], [1, 2, 3], [1, 2, 2], [3, 0, 0], [3, 0, 1], [3, 0, 3], [3, 0, 2], [3, 1, 0], [3, 1, 1], [3, 1, 3], [3, 1, 2], [3, 3, 0], [3, 3, 1], [3, 3, 3], [3, 3, 2], [3, 2, 0], [3, 2, 1], [3, 2, 3], [3, 2, 2], [2, 0, 0], [2, 0, 1], [2, 0, 3], [2, 0, 2], [2, 1, 0], [2, 1, 1], [2, 1, 3], [2, 1, 2], [2, 3, 0], [2, 3, 1], [2, 3, 3], [2, 3, 2], [2, 2, 0], [2, 2, 1], [2, 2, 3], [2, 2, 2]]
